# Notebook 1: Data Loading into PostgreSQL

**Project:** Creating local LLM  
**Author:** Muhammad Ali Tahir  
**Date:** 2025  

---

## 📋 Notebook Objectives

1. **Creating PostSQL DB from dataset


## 1. Environment Setup & Configuration

In [6]:
"""
Step 1: Load Amazon Fine Food Reviews into PostgreSQL
=====================================================
This script:
1. Creates the database schema (products, users, reviews tables)
2. Loads Reviews.csv into PostgreSQL
3. Normalizes data into relational tables
"""

import pandas as pd
from sqlalchemy import create_engine, text
import time
from urllib.parse import quote_plus

# ── Configuration ──────────────────────────────────────────────

DB_USER = "llmuser"
DB_PASS = quote_plus("U9797013u@")  # This encodes the @ symbol
DB_HOST = "127.0.0.1"
DB_PORT = "5432"
DB_NAME = "llmdb"
CSV_PATH = "/home/maliciit/Downloads/archive/Reviews.csv"  # Update path if needed

# ── Connect to Database ───────────────────────────────────────
print("Connecting to PostgreSQL...")
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# ── Create Schema ─────────────────────────────────────────────
print("Creating schema...")
with engine.connect() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
    
    # Drop existing tables (for re-runs)
    conn.execute(text("DROP TABLE IF EXISTS reviews CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS products CASCADE;"))
    conn.execute(text("DROP TABLE IF EXISTS users CASCADE;"))

    # Products table
    conn.execute(text("""
        CREATE TABLE products (
            product_id VARCHAR(20) PRIMARY KEY,
            review_count INTEGER DEFAULT 0,
            avg_score NUMERIC(3,2) DEFAULT 0
        );
    """))

    # Users table
    conn.execute(text("""
        CREATE TABLE users (
            user_id VARCHAR(50) PRIMARY KEY,
            profile_name VARCHAR(255)
        );
    """))

    # Reviews table (main table with text for embeddings)
    conn.execute(text("""
        CREATE TABLE reviews (
            id SERIAL PRIMARY KEY,
            original_id INTEGER,
            product_id VARCHAR(20) REFERENCES products(product_id),
            user_id VARCHAR(50) REFERENCES users(user_id),
            helpfulness_numerator INTEGER,
            helpfulness_denominator INTEGER,
            score INTEGER,
            review_time BIGINT,
            summary TEXT,
            review_text TEXT,
            embedding vector(384)
        );
    """))

    # Indexes for fast retrieval
    conn.execute(text("CREATE INDEX idx_reviews_product ON reviews(product_id);"))
    conn.execute(text("CREATE INDEX idx_reviews_user ON reviews(user_id);"))
    conn.execute(text("CREATE INDEX idx_reviews_score ON reviews(score);"))

    conn.commit()
    print("Schema created successfully!")

# ── Load CSV ──────────────────────────────────────────────────
print(f"\nLoading CSV from {CSV_PATH}...")
start = time.time()
df = pd.read_csv(CSV_PATH)
print(f"Loaded {len(df):,} rows in {time.time() - start:.1f}s")

# Show basic info
print(f"\nColumns: {list(df.columns)}")
print(f"Shape: {df.shape}")
print(f"\nSample row:")
print(df.iloc[0])

# ── Clean Data ────────────────────────────────────────────────
print("\nCleaning data...")
# Drop rows with missing critical fields
df = df.dropna(subset=["ProductId", "UserId", "Text"])
# Remove duplicates based on UserId + ProductId + Time
df = df.drop_duplicates(subset=["UserId", "ProductId", "Time"])
print(f"After cleaning: {len(df):,} rows")

# ── Insert Products ───────────────────────────────────────────
print("\nInserting products...")
products = df.groupby("ProductId").agg(
    review_count=("Id", "count"),
    avg_score=("Score", "mean")
).reset_index()

products.columns = ["product_id", "review_count", "avg_score"]
products["avg_score"] = products["avg_score"].round(2)

start = time.time()
products.to_sql("products", engine, if_exists="append", index=False, method="multi", chunksize=5000)
print(f"Inserted {len(products):,} products in {time.time() - start:.1f}s")

# ── Insert Users ──────────────────────────────────────────────
print("\nInserting users...")
users = df[["UserId", "ProfileName"]].drop_duplicates(subset=["UserId"]).copy()
users.columns = ["user_id", "profile_name"]

# Truncate long profile names
users["profile_name"] = users["profile_name"].astype(str).str[:255]

start = time.time()
users.to_sql("users", engine, if_exists="append", index=False, method="multi", chunksize=5000)
print(f"Inserted {len(users):,} users in {time.time() - start:.1f}s")

# ── Insert Reviews ────────────────────────────────────────────
print("\nInserting reviews...")
reviews = df[["Id", "ProductId", "UserId", "HelpfulnessNumerator",
              "HelpfulnessDenominator", "Score", "Time", "Summary", "Text"]].copy()
reviews.columns = ["original_id", "product_id", "user_id", "helpfulness_numerator",
                    "helpfulness_denominator", "score", "review_time", "summary", "review_text"]

start = time.time()
reviews.to_sql("reviews", engine, if_exists="append", index=False, method="multi", chunksize=5000)
print(f"Inserted {len(reviews):,} reviews in {time.time() - start:.1f}s")

# ── Verify ────────────────────────────────────────────────────
print("\n" + "=" * 50)
print("VERIFICATION")
print("=" * 50)
with engine.connect() as conn:
    for table in ["products", "users", "reviews"]:
        result = conn.execute(text(f"SELECT COUNT(*) FROM {table}"))
        count = result.scalar()
        print(f"  {table}: {count:,} rows")

    # Sample review
    result = conn.execute(text("""
        SELECT r.summary, r.score, LEFT(r.review_text, 100) as text_preview
        FROM reviews r LIMIT 1
    """))
    row = result.fetchone()
    print(f"\nSample review:")
    print(f"  Summary: {row[0]}")
    print(f"  Score: {row[1]}")
    print(f"  Text: {row[2]}...")

print("\n✅ Database loaded successfully!")
print("Next step: Generate embeddings (Step 2)")

Connecting to PostgreSQL...
Creating schema...
Schema created successfully!

Loading CSV from /home/maliciit/Downloads/archive/Reviews.csv...
Loaded 568,454 rows in 2.9s

Columns: ['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text']
Shape: (568454, 10)

Sample row:
Id                                                                        1
ProductId                                                        B001E4KFG0
UserId                                                       A3SGXH7AUHU8GW
ProfileName                                                      delmartian
HelpfulnessNumerator                                                      1
HelpfulnessDenominator                                                    1
Score                                                                     5
Time                                                             1303862400
Summary                                         